In [34]:
from tensorflow.keras.layers import LeakyReLU,Conv2DTranspose,Embedding,Input,Dense,Flatten,Conv2D,Concatenate,MaxPooling2D,Reshape
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.optimizers import Adam
import numpy as np

### Tokenize

In [13]:
dataset_labels_en = {
    0 : "closed fist",
    1 : "finger circles",
    2 : "finger symbols",
    3 : "semi finger bend",
    4 : "open palm",
    5 : "semi open fist",
    6 : "semi open palm",
    7 : "single finger bend",
}

dataset_labels_tr = {
    0 : "kapalı yumruk",
    1 : "parmak çevreler",
    2 : "parmak semboller",
    3 : "yarım parmak bükme",
    4 : "açık avuç",
    5 : "yarım açık yumruk",
    6 : "yarım açık avuç",
    7 : "tek parmak bükme",
}

"""
dataset labels'ın tokenize edilip vektör haline getirilmiş türü
(0, array([4, 9, 0], dtype=int64))
(1, array([5,10, 0], dtype=int64))
(2, array([5, 6, 0], dtype=int64))
(3, array([8, 5, 3], dtype=int64))
(4, array([2, 1, 0], dtype=int64))
(5, array([8, 2, 9], dtype=int64))
(6, array([8, 2, 1], dtype=int64))
(7, array([7, 5, 3], dtype=int64))
"""

In [17]:
concated = ""
for i in dataset_labels_tr.values():
     concated+=f"{i} "
    
print(concated)
kelime_sayilari = Counter(concated.split(" "))

sozluk = []
for kelime, sayi in kelime_sayilari.items():
    sozluk.append(kelime)
    print(f"'{kelime}' kelimesi {sayi} kez geçiyor.")

kapalı yumruk parmak çevreler parmak semboller yarım parmak bükme açık avuç yarım açık yumruk yarım açık avuç tek parmak bükme 
'kapalı' kelimesi 1 kez geçiyor.
'yumruk' kelimesi 2 kez geçiyor.
'parmak' kelimesi 4 kez geçiyor.
'çevreler' kelimesi 1 kez geçiyor.
'semboller' kelimesi 1 kez geçiyor.
'yarım' kelimesi 3 kez geçiyor.
'bükme' kelimesi 2 kez geçiyor.
'açık' kelimesi 3 kez geçiyor.
'avuç' kelimesi 2 kez geçiyor.
'tek' kelimesi 1 kez geçiyor.
'' kelimesi 1 kez geçiyor.


In [16]:
from sklearn.preprocessing import LabelEncoder

In [25]:
encoder = LabelEncoder()
encoded = encoder.fit_transform(sozluk)
print(sozluk)
print(encoded)
words_encoded_dict = {}

for idx in range(len(encoded)):
    words_encoded_dict.update({sozluk[idx]:encoded[idx]})
    
print("\n")
print(words_encoded_dict)

['kapalı', 'yumruk', 'parmak', 'çevreler', 'semboller', 'yarım', 'bükme', 'açık', 'avuç', 'tek', '']
[ 4  9  5 10  6  8  3  2  1  7  0]


{'kapalı': 4, 'yumruk': 9, 'parmak': 5, 'çevreler': 10, 'semboller': 6, 'yarım': 8, 'bükme': 3, 'açık': 2, 'avuç': 1, 'tek': 7, '': 0}


In [42]:
def tokenize():
    tokenized_labels = {}
    for key, value in dataset_labels_tr.items():
        row_encodes = []
        words = value.split(" ")
        for label_word in words:
            row_encodes.append(words_encoded_dict[label_word])
        if(len(row_encodes) != 3):
            row_encodes.append(0)
        tokenized_labels[key] =np.array(row_encodes)
    return tokenized_labels

tokenized_labels = tokenize()

In [43]:
for i in tokenized_labels.items():
    print(i)

(0, array([4, 9, 0], dtype=int64))
(1, array([ 5, 10,  0], dtype=int64))
(2, array([5, 6, 0], dtype=int64))
(3, array([8, 5, 3], dtype=int64))
(4, array([2, 1, 0], dtype=int64))
(5, array([8, 2, 9], dtype=int64))
(6, array([8, 2, 1], dtype=int64))
(7, array([7, 5, 3], dtype=int64))


### Embedding 

In [35]:
model = Sequential()
model.add(Embedding(1,50,input_length=3))
model.compile("rmsprop","mse")

In [36]:
example_seed = np.random.randint(1,3,size=(1,3))
print("example:\n",example_seed,"\n")

example_res = model.predict(example_seed)
print("example output: \n",example_res,"\n")
print(example_res.shape)

example:
 [[2 2 1]] 

1/1 [==============================] - 0s 99ms/step
example output: 
 [[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0.]]] 

(1, 3, 50)


In [61]:
model = Sequential()
model.add(Input((3,)))
model.add(Embedding(10,50,input_length=3))
model.compile("rmsprop","mse")

In [62]:
output = model.predict(example_seed)
output.shape

1/1 [==============================] - 0s 30ms/step


(1, 3, 50)

In [63]:
output = model.predict(tokenized_labels[0])
print("length: ",len(output))
for i in output:
    print("length:",len(i))
print(output)

1/1 [==============================] - 0s 32ms/step
length:  3
length: 50
length: 50
length: 50
[[-0.03849635  0.03954578  0.01342502 -0.04930233  0.02586     0.03696546
  -0.00502933 -0.02727122  0.01802484  0.00921071 -0.01699233 -0.02792304
  -0.0331637  -0.03273916 -0.02591754 -0.02299308 -0.02854205 -0.01010298
   0.00634484  0.00515012  0.03256046  0.04926511  0.04228142  0.03852065
  -0.01402365  0.03936068  0.01543445 -0.03016212  0.00110907 -0.0409855
  -0.0083462  -0.0414372  -0.03168894  0.03096949  0.02557183  0.01507549
  -0.04032337  0.02773463  0.02200239  0.03571426 -0.01204234  0.02130717
   0.0111777  -0.04082455  0.04818416 -0.04067886  0.03065525  0.01547494
   0.01205338  0.03455104]
 [ 0.02086557  0.00388763 -0.02324618 -0.00811727 -0.04210348  0.02063737
   0.02681516  0.04960001  0.03149012  0.0483599  -0.04126732  0.01434969
  -0.0430064  -0.03778229  0.04867173 -0.03521425 -0.00888277 -0.02479272
  -0.04329189 -0.00428547  0.0291196  -0.02228613 -0.00103439 -0

### Model

#### Discriminator

In [71]:
inp_size = (160,160,3)

In [177]:
def build_discriminator(in_shape):
    inp_label = Input((3,))
    embedding = Embedding(1,50,input_length=3)(inp_label)
    d1 = Dense(in_shape[0]*in_shape[1])(embedding)
    reshape1 = Reshape(in_shape)(d1)
    
    inp_image = Input(in_shape)
    concat1 = Concatenate()([reshape1,inp_image])
    
    conv1 = Conv2D(128,(3,3),strides=(2,2))(concat1)
    conv2 = Conv2D(128,(3,3),strides=(1,1))(conv1)
    conv3 = Conv2D(128,(5,5),strides=(2,2))(conv2)
    conv4 = Conv2D(128,(3,3),strides=(2,2))(conv3)
    conv5 = Conv2D(128,(3,3),strides=(2,2))(conv4)
    
    flat1 = Flatten()(conv5)
    dense_output = Dense(1)(flat1)
    
    opt = Adam(learning_rate=0.0002,beta_1=0.5)
    model = Model([inp_label,inp_image],dense_output)
    model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
    
    model.summary()
    
    return model

In [178]:
build_discriminator(inp_size)

Model: "model_59"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_140 (InputLayer)         [(None, 3)]          0           []                               
                                                                                                  
 embedding_64 (Embedding)       (None, 3, 50)        50          ['input_140[0][0]']              
                                                                                                  
 dense_70 (Dense)               (None, 3, 25600)     1305600     ['embedding_64[0][0]']           
                                                                                                  
 reshape_47 (Reshape)           (None, 160, 160, 3)  0           ['dense_70[0][0]']               
                                                                                           

#### Generator

In [179]:
def build_generator(latent_dim):
    inp_label = Input((3,))
    embedding = Embedding(1,50,input_length=3)(inp_label)
    flat1 = Flatten()(embedding)
    
    inp_image_latent = Input((150))
    concated = Concatenate()([flat1,inp_image_latent])
    d1 = Dense(10*10*20)(concated)
    reshape1 = Reshape((10,10,20))(d1)
    
    conv1 = Conv2DTranspose(72,(3,3),strides=(2,2),padding='same')(reshape1)
    conv1 = LeakyReLU(alpha=0.2)(conv1)
    conv2 = Conv2DTranspose(72,(5,5),strides=(2,2),padding='same')(conv1)
    conv2 = LeakyReLU(alpha=0.2)(conv2)
    conv3 = Conv2DTranspose(72,(3,3),strides=(2,2),padding='same')(conv2)
    conv3 = LeakyReLU(alpha=0.2)(conv3)
    conv4 = Conv2DTranspose(3,(3,3),strides=(2,2),padding='same')(conv3)
    conv4 = LeakyReLU(alpha=0.2)(conv4)
    
    model = Model([inp_label,inp_image_latent],conv4)
    model.compile("rmsprop","mse")
    model.summary()
    
    return model
    

In [180]:
build_generator(50)

Model: "model_60"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_142 (InputLayer)         [(None, 3)]          0           []                               
                                                                                                  
 embedding_65 (Embedding)       (None, 3, 50)        50          ['input_142[0][0]']              
                                                                                                  
 flatten_44 (Flatten)           (None, 150)          0           ['embedding_65[0][0]']           
                                                                                                  
 input_143 (InputLayer)         [(None, 150)]        0           []                               
                                                                                           

In [183]:
def build_gan(model_gen,model_disc):
    model_disc.trainable = False
    gen_label,  gen_noise= model_gen.input
    gen_output = model_gen.output
    gan_output = model_disc([gen_label,gen_output])
    
    model = Model([gen_label,gen_noise], gan_output)
    
    opt = Adam(lr=0.0002,beta_1=0.5)
    model.compile(loss='binary_crossentropy',optimizer=opt)
    return model

In [184]:
gan_model = build_gan(build_generator(500),build_discriminator(inp_size))

Model: "model_63"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_148 (InputLayer)         [(None, 3)]          0           []                               
                                                                                                  
 embedding_68 (Embedding)       (None, 3, 50)        50          ['input_148[0][0]']              
                                                                                                  
 flatten_47 (Flatten)           (None, 150)          0           ['embedding_68[0][0]']           
                                                                                                  
 input_149 (InputLayer)         [(None, 150)]        0           []                               
                                                                                           

In [ ]:
def load_real_samples:
    """
    dataset X'in scale ve dimension preprocess'lerinin yapılması
    dataset y ile geri döndürülmesi
    """
    pass

def generate_real_samples:
    """
    dataset'ten rastgele bir görüntü alır ve şu şekilde döndürür
    [X,labels] , y
    X: seçilen görüntü
    labels: seçilen görüntünün etiketi
    y: np.ones(), birlerden oluşan etiket
    """
    pass

In [ ]:
def generate_latent():
    """
    geri döndür -> return( z_input,labels)
    z_input : latent (rastgele) vektörü
    labels : rastgele bir label
    """
    pass

def generate_fake_samples():
    """
    z_input , labels_input = generate_latent() metodunu çağır
    
    images = generator.predicr(z_input,labels_input)
    
    y = zeros() : 0 etiketi
    
    return [images, labels_input], y
    """

In [ ]:
def train():
    """
    generate_real_samples()'tan gerçek image,label,ve etiketi al(ones)
    discriminatoru bununla eğit
    
    generate_fake_samples()'tan sahte image label ve etiketi al(zeros)
    disciminatoru bununla eğit
    
    generate_latent()'ten input ve label rastgele girdilerini al
    y_gan = ones() al
    
    gan modelini son gelen 3 değerle eğit
    
    """
    pass